In [7]:
import mediapipe as mp
import cv2
import pandas as pd
import numpy as np
import csv

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [8]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(
    static_image_mode=True, min_detection_confidence=0.3, model_complexity=2
)

In [9]:
#functions to write and append rows to our csv file
def write_row(row, mode='w'):
    with open(
        "C:/Users/cosmo/Projects/YogaPoseHelper/app/data/yogaposes.csv",
        mode=mode,
        newline="",
    ) as f:
        writer = csv.writer(f, delimiter=",", quotechar='"', quoting=csv.QUOTE_MINIMAL)
        writer.writerow(row)

def append_row(row):
    write_row(mode='a')


def extract_coords_as_row(frame,pose):
    """
    returns pose coordinates in the form : [x0,y0,z0,x1,y1,z1,x2,y2,z2,x3,y3....]
    where x0,y0,z0,x1,y1,z1,x2,y2,z2,x3,y3 are the coordinates of landmarks 0,1,2,3...
    """

    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = pose.process(img)

    if results.pose_landmarks:
        try:
            landmarks = results.pose_landmarks.landmark
            row = list(
                np.array(
                    [[landmark.x, landmark.y, landmark.z] for landmark in landmarks]
                ).flatten()
            )

            return row

        except Exception as e:
            print(e)
            print("Error while exporting coords")
    
    return None

In [13]:
df = pd.read_csv('C:\\Projects\\YogaPoseHelper\\app\\data\\yogaposes.csv')
df.head()

,class,x0,y0,z0,x1,y1,z1,x2,y2,z2,...,z29,x30,y30,z30,x31,y31,z31,x32,y32,z32
0,downdog,0.428334,0.714571,-0.092104,0.407540,0.711688,-0.142556,0.405720,0.706510,-0.143095,...,-0.023344,0.926697,0.807663,0.452329,0.820578,0.855747,-0.235409,0.807258,0.841764,0.316169
1,downdog,0.363939,0.701190,-0.073662,0.342542,0.691830,-0.125583,0.341759,0.687704,-0.126044,...,-0.050904,0.903103,0.852747,0.473810,0.775711,0.890205,-0.248567,0.772837,0.880770,0.335988
2,downdog,0.402440,0.684227,-0.091350,0.373917,0.691096,-0.143665,0.369842,0.685471,-0.144182,...,0.028870,0.930974,0.757986,0.519451,0.838917,0.848080,-0.143772,0.820790,0.811300,0.399058
3,downdog,0.605187,0.700405,-0.034658,0.622995,0.707518,-0.018905,0.625417,0.703752,-0.019291,...,0.260498,0.290139,0.914113,-0.016242,0.384333,0.917647,0.149264,0.379344,0.925649,-0.142140
4,downdog,0.452793,0.780361,-0.044919,0.440372,0.788308,-0.068541,0.439558,0.786809,-0.068853,...,-0.008969,0.712263,0.842772,0.188376,0.654329,0.899086,-0.109187,0.651116,0.884216,0.114346


In [14]:
X = df.drop('class',axis=1)
y = df['class']

X_train,X_test, y_train,y_test = train_test_split(X,y,test_size=0.2)

In [15]:
clf = RandomForestClassifier()

model = clf.fit(X_train,y_train)

testing our model:

In [16]:
row = extract_coords_as_row(cv2.imread('C:\\Users\\cosmo\\Projects\\YogaPoseHelper\\app\\data\\kaggle\\DATASET\\TEST\\plank\\00000001.jpg'),pose)
print(clf.predict(pd.DataFrame(columns=df.columns[1:],data=[row])))
print(clf.score(X_test,y_test))

error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


# Live Preview:

(also full body should be visible in webcam for accurate classification)

In [17]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    success, frame = cap.read()
    pose_name = "No pose detected"

    if success:
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        results = pose.process(img)

        if results.pose_landmarks:
            # predict the pose name
            row = extract_coords_as_row(frame, pose)
            pose_name = clf.predict(pd.DataFrame(columns=df.columns[1:],data=[row]))[0]

            # draw the landmarks
            mp_drawing.draw_landmarks(
                image=img,
                landmark_list=results.pose_landmarks,
                connections=mp_pose.POSE_CONNECTIONS,
            )

            # make the final image equal to the image we drew on
            frame = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    cv2.putText(frame, pose_name, (5, 20), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 2)
    cv2.imshow("webcam feed", frame)

    if cv2.waitKey(10) & 0xFF == ord("q"):
        break


cap.release()
cv2.destroyAllWindows()